# SageMaker-Tensorflow-Hans-on
## ハンズオン構成
![architecture](documents/architecture.svg)

- 概要
    - 手元のPC(mac/linuxを想定)にてPythonスクリプトを使用して、MNISTデータセットの分類モデルをトレーニングし、その後に推論を実行します。
    - コンテナは、SageMaker提供のコンテナを利用します。
    - 最初に`src/training_and_save.py`を実行し、トレーニングとトレーニング済みモデルを利用し推論エンドポイントを準備します。
    - その後`src/invoke_inference.py`を利用し作成した推論用エンドポイントを利用し推論を実行します。

- 元のsampleからの違い
    - ベースは、[amazon-sagemaker-examples](https://github.com/aws/amazon-sagemaker-examples/)の[ensorflow_script_mode_training_and_serving](https://github.com/aws/amazon-sagemaker-examples/blob/main/sagemaker-python-sdk/tensorflow_script_mode_training_and_serving/tensorflow_script_mode_training_and_serving.ipynb)(日本語は[こちら](https://github.com/aws-samples/aws-ml-jp/blob/main/sagemaker/tensorflow2-training-and-serving/tensorflow2_training_and_serving.ipynb))です
    - 元はSageMakerのnotebookやSageMaker Labでの実行を前提としていますが、本ハンズオンは手元のPCでnotebookを使用せず実行する前提のハンズオンにしています。
    - 元のsampleからの変更点は以下になります
        - SageMaker実行ロール: 元は作業環境のロールをそのまま利用する前提のコードですが、本ハンズオンは作業用のロールとSageMakerの実行ロールを分ける前提で実装しています。
        - 学習と推論のスクリプト分離: 学習と推論を別々に実行する前提で、スクリプトを分けました。

## Step1: 事前準備
### 実行環境の確認
作業環境に以下のソフトウェアがセットアップされていることを確認します。ない場合はインストールします。
- python3
- pip3(sagemakerのセットアップ用)
- AWS CLI(事前の動作確認用)


SageMakerを実行したい環境用のプロファイルがあることを確認します。ここではプロファイル名に`sagemaker-poc-profile`という名前を利用していると仮定します。
以下の例のようにUserIdやアカウントIDが取得できれば問題ありません。
なおこの実行ロールには、`AdministratorAccess`相当の権限が付与されている前提とします(SageMaker用の実行ロール作成のため)。

In [43]:
%%bash
PROFILE=sagemaker-poc-profile
echo "PROFILE=${PROFILE}" > environment
echo "export PROFILE" >> environment

In [44]:
%%bash
. ./environment 
aws --profile ${PROFILE} sts get-caller-identity

{
    "UserId": "AROAQH6XODSXUFCH7NNDU:botocore-session-1676978959",
    "Account": "017094810799",
    "Arn": "arn:aws:sts::017094810799:assumed-role/OrganizationAccountAccessRole/botocore-session-1676978959"
}


下記のようにアカウントIDやARNが表示されればOKです。
```
{
    "UserId": "AROAQH6XODSXUFCH7NNDU:botocore-session-1676905613",
    "Account": "999999999999",
    "Arn": "arn:aws:sts::999999999999:assumed-role/OrganizationAccountAccessRole/botocore-session-1676905613"
}
```
### SageMaker用の実行ロールとS3バケットの作成
![architecture](documents/step1-1.svg)

In [45]:
%%bash
. ./environment
aws --profile ${PROFILE} cloudformation deploy \
    --stack-name SageMakerPoC \
    --template-file "./CFn/execute_role.yaml" \
    --capabilities CAPABILITY_IAM ;


Waiting for changeset to be created..

No changes to deploy. Stack SageMakerPoC is up to date


### python仮想環境の準備
今回は手元のPCで実行するため、PC環境にSageMaker SDKが残らないように、ハンズオン用のpython仮想環境を準備します。これはハンズオン固有の設定になるため、SageMakerの利用上は必要はありません。

In [46]:
%%bash
#python仮想環境の作成
python3 -m venv sagemaker-hans-on

#python仮想環境のアクティブ化
source sagemaker-hans-on/bin/activate

### 必要なPythonライブラリのセットアップ

In [ ]:
%%bash
pip3 install sagemaker matplotlib

### 事前作成したIAMロールとS3バケット情報の設定
トレーニング&推論実行時に引き渡すために、SageMakerの実行ロールとデフォルトS3バケットの情報を環境変数に設定します。

In [48]:
%%bash
. ./environment
#ClientのPrivate IP取得
ExecuteRoleArn=$(aws --profile ${PROFILE} --output text \
    cloudformation describe-stacks \
        --stack-name SageMakerPoC \
        --query 'Stacks[].Outputs[?OutputKey==`RoleArn`].[OutputValue]')
BucketName=$(aws --profile ${PROFILE} --output text \
    cloudformation describe-stacks \
        --stack-name SageMakerPoC \
        --query 'Stacks[].Outputs[?OutputKey==`BucketName`].[OutputValue]')
# environmentファイルへの追加
echo "ExecuteRoleArn=${ExecuteRoleArn}" >> environment
echo "BucketName=${BucketName}" >> environment
echo "export ExecuteRoleArn BucketName" >> environment

設定確認

In [49]:
%%bash
. ./environment
env | grep -e ExecuteRoleArn -e BucketName

BucketName=sagemaker-ap-northeast-1-017094810799
ExecuteRoleArn=arn:aws:iam::017094810799:role/SageMakerPoC-SageMakerExecuteRole-1CXV2KBCJBON6


### (オプション)トレーニングデータの確認
![architecture](documents/step1-2.svg)
デモ用のトレーニングデータとして、SageMakerがデモ用にS3にてPublicで提供しているMNISTデータベースを利用します。
ここでMNISTデータベースのバケットの中身を確認します。

In [1]:
%%bash
. ./environment
aws --profile ${PROFILE} s3 ls s3://sagemaker-sample-data-ap-northeast-1/tensorflow/mnist/

2019-01-25 08:31:41   31360128 eval_data.npy
2019-01-25 08:31:41      40128 eval_labels.npy
2019-01-25 08:31:41  172480128 train_data.npy
2019-01-25 08:31:41     220128 train_labels.npy


## Step2: トレーニングの実行と推論エンドポイントの作成
### トレーニングの実行/モデル保存/推論エンドポイント作成
![architecture](documents/step2-1.svg)

In [51]:
%%bash
. ./environment
python3 src/training_and_save.py

region            = ap-northeast-1
execute_role      = arn:aws:iam::017094810799:role/SageMakerPoC-SageMakerExecuteRole-1CXV2KBCJBON6
default_bucket    = sagemaker-ap-northeast-1-017094810799
training_data_url = s3://sagemaker-sample-data-ap-northeast-1/tensorflow/mnist


INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: tensorflow-training-2023-02-21-12-00-11-831


### 結果の確認と環境変数の設定

In [ ]:
%%bash
. ./environment

#手動入力
# JOB_NAME="<training_and_save.py実行最後に出力されたJob Name>"
# ENDPOINT_NAME="<training_and_save.py実行最後に出力されたEndpoint Name"
JOB_NAME="tensorflow-training-2023-02-21-11-39-34-363"
ENDPOINT_NAME="tensorflow-training-2023-02-21-11-44-20-243"

#確認
echo "トレーニング済みモデルデータの確認"
aws --profile ${PROFILE} s3 ls "s3://${BucketName}/${JOB_NAME}/output/"

echo
echo "エンドポイントの確認"
aws --profile ${PROFILE} sagemaker describe-endpoint --endpoint-name tensorflow-training-2023-02-21-11-44-20-243 --query '[EndpointName,EndpointStatus]'

#環境変数の設定
echo "JOB_NAME=${JOB_NAME}" >> environment
echo "ENDPOINT_NAME=${ENDPOINT_NAME}" >> environment
echo "export JOB_NAME ENDPOINT_NAME" >> environment

トレーニング済みモデルデータの確認
2023-02-21 20:43:51    8143276 model.tar.gz

エンドポイントの確認
[
    "tensorflow-training-2023-02-21-11-44-20-243",
    "InService"
]


## Step3: 推論の実行
### 推論用のデータをローカルに取得
![architecture](documents/step3-1.svg)

In [ ]:
%%bash
. ./environment

#ディレクトリの作成
mkdir inference
cd inference

#推論の入力データのダウンロード
aws --profile ${PROFILE} s3 \
    cp s3://sagemaker-sample-data-ap-northeast-1/tensorflow/mnist/eval_data.npy eval_data.npy
aws --profile ${PROFILE} s3 \
    cp s3://sagemaker-sample-data-ap-northeast-1/tensorflow/mnist/eval_labels.npy eval_labels.npy

download: s3://sagemaker-sample-data-ap-northeast-1/tensorflow/mnist/eval_data.npy to ./eval_data.npy
download: s3://sagemaker-sample-data-ap-northeast-1/tensorflow/mnist/eval_labels.npy to ./eval_labels.npy


### 推論の実行

In [ ]:
%%bash
. ./environment

cd inference
python3 ../src/invoke_inference.py

## Step4: クリーンナップ
### 推論エンドポイントの削除

In [ ]:
%%bash
. ./environment

cd inference
python3 ../src/remove_inference_endpoint.py

endpoint_name      = tensorflow-training-2023-02-21-11-44-20-243


### ローカルデータの削除

In [ ]:
%%bash
rm -rf inference

### python仮想環境の終了

In [ ]:
%%bash
deactivate